# PageRank на Spark RDD

### Шаг №1
Создайте SparkContext

In [1]:
import os
import sys

SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [2]:
import random
from pyspark import SparkContext, SparkConf

spark_ui_port = random.choice(range(10000, 11000))
print(f"Spark UI port: {spark_ui_port}")

conf = SparkConf()
conf.set("spark.ui.port", spark_ui_port)

sc = SparkContext(appName="Pagerank", conf=conf)

Spark UI port: 10298


### Шаг №2
1. Прочитайте граф из файла `/datasets/spark/graph.tsv`
2. Создайте RDD, в которой граф будет представлен парами вершин
3. Убедитесь, что граф совпадает с рисунком на доске

In [22]:
raw_graph = sc.textFile('/datasets/spark/graph.tsv')
raw_graph.collect()
graph = raw_graph.map(lambda x: x.split('\t')).cache()
graph.collect()

[['A', 'B'],
 ['A', 'C'],
 ['B', 'D'],
 ['C', 'A'],
 ['C', 'B'],
 ['C', 'D'],
 ['D', 'C']]

### Шаг №3
Создайте RDD с первоначальными pagerank всех уникальных вершин

In [23]:
vertices = graph.map(lambda x: x[0]).union(graph.map(lambda x: x[0])).distinct()

In [24]:
vertices.collect()

['A', 'B', 'C', 'D']

In [25]:
num_vertices = vertices.count()

In [26]:
pagerank = vertices.map(lambda x: (x, 1 / num_vertices))
pagerank.collect()

[('A', 0.25), ('B', 0.25), ('C', 0.25), ('D', 0.25)]

### Шаг №4
Создайте RDD, которая берет RDD с вершинами, объединяет ее с RDD с pagerank. В результате должна получится PairRDD, где ключ - это уникальная вершина, а значение - это все вершины, на которые она ссылаются и ее текущий pagerank

In [27]:
links = graph.groupByKey().mapValues(list).cache()

In [28]:
contributions = links.join(pagerank)
contributions.collect()

[('C', (['B', 'D', 'A'], 0.25)),
 ('A', (['B', 'C'], 0.25)),
 ('B', (['D'], 0.25)),
 ('D', (['C'], 0.25))]

### Шаг №5
Реализуйте функцию, которая рассчитывает pagerank для всех вершин, на которые ссылается данная вершина. Функция должна быть итератором, который возвращает вершину и ее pagerank

In [29]:
def pagerank_elements(neighbours, pagerank):
    n = len(neighbours)
    for i in neighbours:
        yield (i, pagerank / n)

### Шаг №6
Обновите RDD с pagerank значениями, посчитанными с помощью функции из предыдущего шага

In [30]:
pagerank = contributions.flatMap(lambda x: pagerank_elements(x[1][0],x[1][1])).reduceByKey(lambda x, y: x + y)
pagerank.collect()

[('C', 0.375),
 ('A', 0.08333333333333333),
 ('B', 0.20833333333333331),
 ('D', 0.3333333333333333)]

### Шаг №7
Напишите цикл, который проводит несколько итераций вычисления pagerank и на каждой печатает номер итерации и текущие pagerank

In [31]:
ITERATIONS = 5

for i in range (ITERATIONS):
    links = graph.groupByKey().mapValues(list).cache()
    contributions = links.join(pagerank)
    pagerank = contributions.flatMap(lambda x: pagerank_elements(x[1][0],x[1][1])).reduceByKey(lambda x, y: x + y)
    print(f"Iter {i} of {ITERATIONS}. Current pagerank: {sorted(pagerank.collect(), key=lambda x: x[0])} ")
pagerank.collect()

Iter 0 of 5. Current pagerank: [('A', 0.125), ('B', 0.16666666666666666), ('C', 0.375), ('D', 0.3333333333333333)] 
Iter 1 of 5. Current pagerank: [('A', 0.125), ('B', 0.1875), ('C', 0.3958333333333333), ('D', 0.29166666666666663)] 
Iter 2 of 5. Current pagerank: [('A', 0.13194444444444445), ('B', 0.19444444444444445), ('C', 0.35416666666666663), ('D', 0.3194444444444444)] 
Iter 3 of 5. Current pagerank: [('A', 0.11805555555555554), ('B', 0.18402777777777776), ('C', 0.38541666666666663), ('D', 0.3125)] 
Iter 4 of 5. Current pagerank: [('A', 0.1284722222222222), ('B', 0.18749999999999997), ('C', 0.3715277777777778), ('D', 0.3125)] 


[('B', 0.18749999999999997),
 ('D', 0.3125),
 ('C', 0.3715277777777778),
 ('A', 0.1284722222222222)]

In [21]:


for i in range (0,5):
    contributions = links.join(pagerank)
    pagerank = contributions.flatMap(lambda x: pagerank_elements(x[1][0],x[1][1])).reduceByKey(lambda x, y: x + y)
    
pagerank.collect()

[('D', 0.31251339591906724),
 ('C', 0.37495579346707825),
 ('A', 0.12502545224622771),
 ('B', 0.18750535836762688)]

### Шаг №8
Не забудьте остановить SparkContext

In [32]:
sc.stop()